In [1]:
import os, sys, gc
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import sqlite3
from shapely import wkt
from shapely.geometry import Polygon, Point
from shapely import make_valid
from shapely.errors import GEOSException

pd.set_option('display.max_columns', 100)
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
def size_sqlite_table(table_name, con) : 
    cursor = con.cursor()
    cursor.execute(f"SELECT * FROM {table_name};")
    results = cursor.fetchall()
    size = len(results)
    del cursor
    _ = gc.collect()
    return size

In [3]:
def gpd_osm_within_wilaya(osm_table_name, wilaya, chunks = 2) : 
    
    # Create connexion with db
    sql_con_wilayats = sqlite3.connect("/home/betorcha/Ml_projects/Osm/data/dz_decoupage.db")
    sql_con_osm = sqlite3.connect("/home/betorcha/Ml_projects/Osm/data/algeria_osm.sqlite")

    # first get the size of table
    table_size = size_sqlite_table(osm_table_name, sql_con_osm)
    print(f"Size of table {osm_table_name} : {table_size}")

    # filter wilaya of interest
    dz = pd.read_sql('select * from all_wilayas;', con = sql_con_wilayats)
    w_geom = dz.loc[dz['name'] == wilaya, 'WKT_GEOMETRY']
    w_geom = wkt.loads(w_geom.values[0])

    # Load chunk of data (virtual memory efficciency)
    
    lim = int(table_size/chunks)
    off = 0
    data = []

    for chunk in range(chunks) : 
        
        df = pd.read_sql(f'select * from {osm_table_name} limit {lim} OFFSET {off};', sql_con_osm)
        #print('df shape', df.shape)

        # is in w_geom?
        geoser = gpd.GeoSeries.from_wkt(df['WKT_GEOMETRY'])
        
        try : 
            mask_w = geoser.within(w_geom)
        except GEOSException as e : 
            print("GEOSException handled : make_valid geometry is ongoing..")
            geoser = geoser.apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
            print("make_valid geometry completed")
            mask_w = geoser.within(w_geom)

        # keep df of wilaya of interest
        df = df[mask_w]
        data.append(df)

        off += int(table_size/chunks)

        if chunk == chunks-2 : 
            lim = table_size - int(table_size/chunks) * (chunks -1) 
        
        print(f"Chunk {chunk} successfuly loaded")

    # convert dataframe into geodataframe
    data = pd.concat(data, axis= 0, ignore_index = True)
    #data['geometry'] = gpd.GeoSeries.from_wkt(data['WKT_GEOMETRY'])
    #data = data.drop('WKT_GEOMETRY', axis = 1)
    #gdf = gpd.GeoDataFrame(data, geometry='geometry')
    #gdf.crs = 4326
#
    #del data, df, geoser, mask_w
    #_ = gc.collect()
    #
    #print(f"geopandas for {wilaya} of shape {gdf.shape} : operation completed!")

    return data

In [5]:
wilaya = 'Djelfa'
tables = ['lines', 'points', 'multilinestrings', 'multipolygons', 'other_relations']

df_wilaya = [] 

for osm_table_name  in tables : 
    df = gpd_osm_within_wilaya(osm_table_name, wilaya)
    df_wilaya.append(df)

Size of table lines : 719364
Chunk 0 successfuly loaded
Chunk 1 successfuly loaded
Chunk 2 successfuly loaded
Size of table points : 486788
Chunk 0 successfuly loaded
Chunk 1 successfuly loaded
Chunk 2 successfuly loaded
Size of table multilinestrings : 470
Chunk 0 successfuly loaded
Chunk 1 successfuly loaded
Chunk 2 successfuly loaded
Size of table multipolygons : 3272288
GEOSException handled : make_valid geometry is ongoing..
make_valid geometry completed
Chunk 0 successfuly loaded
Chunk 1 successfuly loaded
Chunk 2 successfuly loaded
Size of table other_relations : 5874
Chunk 0 successfuly loaded
Chunk 1 successfuly loaded
Chunk 2 successfuly loaded


In [ ]:
df_wilaya = pd.concat(df_wilaya, ignore_index=True)
df_wilaya.head()

In [15]:
conn = sqlite3.connect(f'/home/betorcha/Ml_projects/Osm/osm_wilayats_sqlite/osm_{wilaya}.db')
df_wilaya.to_sql(f'osm_{wilaya}', conn, if_exists = 'replace')

28945

In [3]:
con_djelfa = sqlite3.connect("/home/betorcha/Ml_projects/Osm/osm_wilayats_sqlite/osm_Djelfa.db")
cursor = con_djelfa.cursor()

query = """
SELECT 
    name
FROM 
    sqlite_master
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
    """

tables = cursor.execute(query)
tables.fetchall()

[('osm_Djelfa',)]

In [5]:
df = pd.read_sql('select * from osm_Djelfa;', con = con_djelfa)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28945 entries, 0 to 28944
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ogc_fid       28945 non-null  int64  
 1   WKT_GEOMETRY  28945 non-null  object 
 2   osm_id        19981 non-null  object 
 3   name          2722 non-null   object 
 4   highway       13051 non-null  object 
 5   waterway      137 non-null    object 
 6   aerialway     0 non-null      object 
 7   barrier       275 non-null    object 
 8   man_made      170 non-null    object 
 9   railway       120 non-null    object 
 10  z_order       14198 non-null  float64
 11  other_tags    12902 non-null  object 
 12  ref           1 non-null      object 
 13  address       0 non-null      object 
 14  is_in         0 non-null      object 
 15  place         115 non-null    object 
 16  type          123 non-null    object 
 17  osm_way_id    8964 non-null   object 
 18  aeroway       16 non-null 